In [1]:
import pandas as pd
import json
from IPython.display import JSON
import preprocessing

### Parse json files containing questions from consumers and experts

In [2]:
filepath = 'data/EPIC-QA/consumer_questions_primary.json'
consumer_df = preprocessing.parse_questions(filepath)
consumer_df.head()

,question_id,question,query,background
0,CQ101,If the common cold is a type of coronavirus an...,coronavirus vaccine landscape,Looking for information on global efforts to s...
1,CQ102,Does adequate sleep prevent COVID-19?,Sleep prevent covid,Looking for any confirmation that better sleep...
2,CQ103,Can I go home when I have COVID-19 and pneumonia?,COVID-19 hospital discharge,looking for information on leaving hospital af...
3,CQ104,What endocrine complications are linked to CO...,endocrine complications covid-19,Looking for information on damage to endocrine...
4,CQ105,How long do COVID-19 antibodies stay in your s...,COVID-19 immunity duration,Looking for information on how long a person w...


In [3]:
# consumer_df.to_csv('data/processed/questions_consumer.csv', sep='\t')

In [4]:
# parse json files containing questions from consumers and experts
filepath = 'data/EPIC-QA/expert_questions_primary.json'
expert_df = preprocessing.parse_questions(filepath)
expert_df.head()

,question_id,question,query,background
0,EQ101,What features of SARS-CoV2 are targeted in vac...,coronavirus vaccine landscape,Seeking information on guiding principles for ...
1,EQ102,How do cytokine pathways link sleep and immuni...,Sleep prevent covid,Looking for information on bidirectional inter...
2,EQ103,When is it safe to discharge a COVID-19 patien...,COVID-19 hospital discharge,Looking for guidelines on discharging COVID-19...
3,EQ104,What endocrine complications are linked to CO...,endocrine complications covid-19,Looking for studies investigating whether SARS...
4,EQ105,What is the duration of protection and immunit...,COVID-19 immunity duration,Looking for longitudinal serological studies t...


In [5]:
# expert_df.to_csv('data/processed/questions_expert.csv', sep='\t')

### Extract titles and abstracts from metadata file

In [2]:
metadata = preprocessing.load_metadata('data/cord-19/metadata.csv')

CSV file loaded successfully
Metadata cleaning complete


In [4]:
# Create table of just cord_uid, title, and abstracts
content = metadata[['cord_uid', 'title', 'abstract']]

In [11]:
# # save as csv
# content.to_csv('data/processed/titles_abstracts.csv', sep='\t')